In [7]:
import pandas as pd

import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate() #method to create spark session

# A. Create a spark data frame that contains your favorite programming languages.

### 1. The name of the column should be language

In [8]:
language = pd.DataFrame({'name': ['python','html','java','swift','c++','javascript']})

language

,name
0,python
1,html
2,java
3,swift
4,c++
5,javascript


In [9]:
language = spark.createDataFrame(language)

### 2. View the schema of the dataframe

In [10]:
language.printSchema()

root
 |-- name: string (nullable = true)



### 3. Output the shape of the dataframe

In [14]:
# no index in spark
language.count()

6

### 4. Show the first 5 records in the dataframe

In [12]:
language.show(5)

+------+
|  name|
+------+
|python|
|  html|
|  java|
| swift|
|   c++|
+------+
only showing top 5 rows



# B. Load the mpg dataset as a spark dataframe.

In [25]:
from pydataset import data

mpg = spark.createDataFrame(data('mpg'))

mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



### 1. Create 1 column of output that contains a message like the one below:
    - The 1999 audi a4 has a 4 cylinder engine.

In [31]:
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit

In [37]:
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), mpg.model, lit(' has a'), mpg.cyl, lit('cylinder engine.')).alias('string')).show(truncate=False)

+------------------------------------------------------------+
|string                                                      |
+------------------------------------------------------------+
|The 1999 audi a4 has a4cylinder engine.                     |
|The 1999 audi a4 has a4cylinder engine.                     |
|The 2008 audi a4 has a4cylinder engine.                     |
|The 2008 audi a4 has a4cylinder engine.                     |
|The 1999 audi a4 has a6cylinder engine.                     |
|The 1999 audi a4 has a6cylinder engine.                     |
|The 2008 audi a4 has a6cylinder engine.                     |
|The 1999 audi a4 quattro has a4cylinder engine.             |
|The 1999 audi a4 quattro has a4cylinder engine.             |
|The 2008 audi a4 quattro has a4cylinder engine.             |
|The 2008 audi a4 quattro has a4cylinder engine.             |
|The 1999 audi a4 quattro has a6cylinder engine.             |
|The 1999 audi a4 quattro has a6cylinder engine.       

### 2. For each vehicle.
    - Transform the trans column so that it only contains either manual or auto.

In [161]:
from pyspark.sql.functions import when, sum

In [46]:
mpg.select(mpg.trans,
          when((mpg.trans.contains('auto')), 'auto')
          .otherwise('manual')
          .alias('trans')).show(20)

+----------+------+
|     trans| trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
|manual(m5)|manual|
|  auto(av)|  auto|
|manual(m5)|manual|
|  auto(l5)|  auto|
|manual(m6)|manual|
|  auto(s6)|  auto|
|  auto(l5)|  auto|
|manual(m5)|manual|
|  auto(s6)|  auto|
|manual(m6)|manual|
|  auto(l5)|  auto|
|  auto(s6)|  auto|
|  auto(s6)|  auto|
|  auto(l4)|  auto|
|  auto(l4)|  auto|
+----------+------+
only showing top 20 rows



# C. Load the tips dataset as a spark dataframe.

In [82]:
tips = spark.createDataFrame(data('tips'))

tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [83]:
tips.count()

244

### 1. What percentage of observations are smokers?

In [124]:
from pyspark.sql.functions import avg, round

In [177]:
print((tips.filter(tips.smoker == 'Yes').count() / tips.count() * 100), 'percent of observations are smokers.')

38.114754098360656 percent of observations are smokers.


### 2. Create a column that contains the tip percentage

In [93]:
from pyspark.sql.functions import col, expr

In [96]:
tips = tips.select(
        tips.total_bill, tips.tip, tips.sex, tips.smoker, tips.day, tips.time, tips.size,
        (tips.total_bill / tips.tip).alias('tip_percent')
)

In [97]:
tips.show(5)

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|       tip_percent|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2| 16.82178217821782|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3| 6.228915662650603|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3| 6.002857142857144|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|7.1540785498489425|
|     24.59|3.61|Female|    No|Sun|Dinner|   4| 6.811634349030471|
+----------+----+------+------+---+------+----+------------------+
only showing top 5 rows



### 3. Calculate the average tip percentage for each combination of sex and smoker.

In [142]:
tips.rollup('sex','smoker').agg(avg(tips.tip_percent)).show()

+------+------+------------------+
|   sex|smoker|  avg(tip_percent)|
+------+------+------------------+
|  Male|  null|  7.23406046621358|
|Female|   Yes| 6.448210654922861|
|  Male|   Yes| 8.117240108903532|
|  null|  null| 7.048932200195785|
|Female|    No| 6.877796519255278|
|  Male|    No|6.6877637800136105|
|Female|  null| 6.714850156922292|
+------+------+------------------+



# D. Use the seattle weather dataset referenced in the lesson to answer the questions below.

### 1. Convert the temperatures to farenheight.

### 2. Which month has the most rain, on average?

### 3. Which year was the windiest?

### 4. What is the most frequent type of weather in January?

### 5. What is the average high and low temperature on sunny days in July in 2013 and 2014?

### 6. What percentage of days were rainy in q3 of 2015?

### 7. For each year, find what percentage of days it rained (had non-zero precipitation).